## Denoising Diffusion Probabilistic Models

In [5]:
!gpustat

n101.mcl.weizmann.ac.il  Sun Mar 13 16:36:11 2022  450.51.05
[0] Tesla V100-PCIE-32GB | 30'C,   0 % |     0 / 32510 MB |
[1] Tesla V100-PCIE-32GB | 29'C,   0 % |     0 / 32510 MB |
[2] Tesla V100-PCIE-32GB | 28'C,   0 % |     4 / 32510 MB |
[3] Tesla V100-PCIE-32GB | 30'C,   0 % |     0 / 32510 MB |
[4] Tesla V100-PCIE-32GB | 28'C,   0 % |     0 / 32510 MB |
[5] Tesla V100-PCIE-32GB | 27'C,   0 % |     0 / 32510 MB |


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from common_utils.notebook_utils import *

device=cuda:0


In [18]:
from torch.utils.data import DataLoader
import torchvision
import imageio
import glob
import pytorch_lightning as pl

from datasets.places365 import Places365
from datasets.diffset import DiffSet
from model import DiffusionModel

### Set model parameters

In [15]:
# Training hyperparameters
diffusion_steps = 1000
dataset_choice = "MNIST"
max_epoch = 10
batch_size = 128

# Loading parameters
load_model = False
load_version_num = 1

### Load dataset and train model

In [16]:
# Code for optionally loading model
pass_version = None
last_checkpoint = None

if load_model:
    pass_version = load_version_num
    last_checkpoint = glob.glob(
        f"./lightning_logs/{dataset_choice}/version_{load_version_num}/checkpoints/*.ckpt"
    )[-1]

In [19]:
# Create datasets and data loaders
if dataset_choice in ['MNIST', 'CIFAR']:
    train_dataset = DiffSet(train=True, root='/net/mraid11/export/vision/datasets/', dataset=dataset_choice)
    val_dataset = DiffSet(train=False, root='/net/mraid11/export/vision/datasets/', dataset=dataset_choice)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
elif dataset_choice == 'PLACES365':
    train_dataset = Places365(train=True, root='/net/mraid11/export/data/nivh/datasets/places365_standard/')
    val_dataset = Places365(train=False, root='/net/mraid11/export/data/nivh/datasets/places365_standard/')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
else:
    raise "Unsupported dataset"

# Create model and trainer
if load_model:
    model = DiffusionModel.load_from_checkpoint(last_checkpoint, in_size=train_dataset.size*train_dataset.size, t_range=diffusion_steps, img_depth=train_dataset.depth)
else:
    model = DiffusionModel(train_dataset.size*train_dataset.size, diffusion_steps, train_dataset.depth)

In [20]:
# Load Trainer model
tb_logger = pl.loggers.TensorBoardLogger(
    "lightning_logs/",
    name=dataset_choice,
    version=pass_version,
)

trainer = pl.Trainer(
    max_epochs=max_epoch, 
    log_every_n_steps=10, 
    gpus=1, 
    auto_select_gpus=True,
    resume_from_checkpoint=last_checkpoint, 
    logger=tb_logger
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [21]:
# Train model
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
Missing logger folder: lightning_logs/MNIST

   | Name  | Type       | Params
--------------------------------------
0  | inc   | DoubleConv | 37.7 K
1  | down1 | Down       | 295 K 
2  | down2 | Down       | 1.2 M 
3  | down3 | Down       | 2.4 M 
4  | up1   | Up         | 6.2 M 
5  | up2   | Up         | 1.5 M 
6  | up3   | Up         | 406 K 
7  | outc  | OutConv    | 65    
8  | sa1   | SAWrapper  | 395 K 
9  | sa2   | SAWrapper  | 395 K 
10 | sa3   | SAWrapper  | 99.6 K
--------------------------------------
12.9 M    Trainable params
0         Non-trainable params
12.9 M    Total params
51.676    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/yanivni/data/bin/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


### Sample from model

In [ ]:
gif_shape = [3, 3]
sample_batch_size = gif_shape[0] * gif_shape[1]
n_hold_final = 10

# Generate samples from denoising process
gen_samples = []
x = torch.randn((sample_batch_size, train_dataset.depth, train_dataset.size, train_dataset.size))
sample_steps = torch.arange(model.t_range-1, 0, -1)
for t in sample_steps:
    x = model.denoise_sample(x, t)
    if t % 50 == 0:
        gen_samples.append(x)
for _ in range(n_hold_final):
    gen_samples.append(x)
gen_samples = torch.stack(gen_samples, dim=0).moveaxis(2, 4).squeeze(-1)
gen_samples = (gen_samples.clamp(-1, 1) + 1) / 2

In [ ]:
# Process samples and save as gif
gen_samples = (gen_samples * 255).type(torch.uint8)
gen_samples = gen_samples.reshape(-1, gif_shape[0], gif_shape[1], train_dataset.size, train_dataset.size, train_dataset.depth)

def stack_samples(gen_samples, stack_dim):
    gen_samples = list(torch.split(gen_samples, 1, dim=1))
    for i in range(len(gen_samples)):
        gen_samples[i] = gen_samples[i].squeeze(1)
    return torch.cat(gen_samples, dim=stack_dim)

gen_samples = stack_samples(gen_samples, 2)
gen_samples = stack_samples(gen_samples, 2)

imageio.mimsave(
    f"{trainer.logger.log_dir}/pred.gif",
    list(gen_samples),
    fps=5,
)